In [2]:
import snsim
import pandas as pd

In [3]:
randseed = 1234
z_range = [0.01, 0.15]

time_range = ["2021-08-03", "2022-08-03"]

snia_gen = {'M0': 'jla',
            'sigM': 0.12,
            'sct_model': 'G10',
            'rate': 1,
            'model_config': {'model_name': 'salt3',
                             'alpha': 0.14,
                             'beta': 2.9,
                             'dist_x1': 'N21',
                             'dist_c': [-0.055, 0.023, 0.150]}}

cosmology = {'name':'planck18'}

cmb = {'v_cmb': 0,
       'l_cmb':0,
       'b_cmb':0}


mw_dust = {'model': 'CCM89',}
host={'host_file': '/renoir/carreres/mydatadec/OuterRim/Mocks/OR_mock_00.parquet',
      'key_dic': {'zcos': 'redshift','vpec_true': 'v_radial'},
      'distrib': 'as_host'}
survey = {
        'survey_file': '/renoir/carreres/mydatadec/MySim/paper_sim/input_files/ztf_obsfile_maglimcat.parquet',
        'sig_psf': 0.0,
        'sig_zp': 0.01,
        'ra_size': 7.295,
        'dec_size': 7.465,
        'noise_key': ['maglimcat', 'mlim5'],
        'sub_field': 'rcid',
        'field_map': '/renoir/carreres/Documents/Paper_script/sim_input/ZTF_CCD.dat'
         }


survey =  snsim.survey_host.SurveyObs(survey)


In [4]:
host = snsim.survey_host.SnHost(host, z_range=z_range, footprint=survey.fields.footprint)

In [ ]:
from astropy.time import Time

# Take the SN Ia parameters generator class
gen_class = getattr(snsim.generators, snsim.generators.__GEN_DIC__['snia_gen'])

# Give the input configuration
SNgenerator = gen_class(snia_gen,
                        cmb,
                        snsim.utils.set_cosmo(cosmology),
                        mw_dust=mw_dust,
                        host=host,
                        survey_footprint=survey.fields.footprint)

# Compute redshift distribution
SNgenerator.compute_zcdf(z_range)

# Set mint time and max time
SNgenerator.time_range = (Time(time_range[0]).mjd, Time(time_range[1]).mjd)



In [ ]:
%%time
n_obj = 10000
# Gen basic parameters
params = SNgenerator.gen_astrobj_par(n_obj, randseed)


In [7]:
import geopandas as gpd
import shapely.geometry as shp_geo

In [ ]:
%%time
n= 100
choice_weights = 1 / (1 + host.table['redshift'])
choice_weights /= choice_weights.sum()
choice_weights = None


idx = []
while len(idx) < n:
    idx_tmp = np.random.choice(host.table.index, p=choice_weights)
    pt = shp_geo.Point((host.table.loc[idx_tmp]['ra'], host.table.loc[idx_tmp]['dec']))
    if host._footprint.contains(pt):
        idx.append(idx_tmp)

In [ ]:
min_x = 0
min_y = 0
max_x = 100
max_y = 100
overestimate = 2
np.random.uniform((min_x, min_y), (max_x, max_y), size=(10))

In [ ]:
host.table.loc[idx_tmp]['ra']

In [ ]:
n=10000

In [ ]:
import numpy as np
z_cdf = SNgenerator.z_cdf


choice_weights = 1 / (1 + host.table['redshift'])
choice_weights /= choice_weights.sum()

idx = np.random.choice(host.table.index, p=choice_weights, size=n_obj)
params_assn = host.table.loc[idx]

In [ ]:
import matplotlib.pyplot as plt
bi, a, b  = plt.hist(params.zcos, bins=30, range=[0.0,0.15])
bi1, a1, b1 = plt.hist(params_assn.redshift, bins=30, range=[0.0, 0.15], histtype='step');
#bi2, a2, b2 = plt.hist(params_assn2.redshift, bins=200, range=[0.0, 0.15], histtype='step');


plt.figure()
plt.plot((a[:-1] + a[1:]) * 0.5, (1 + (a[:-1] + a[1:]) * 0.5) * 0.9)
plt.plot((a[:-1] + a[1:]) * 0.5, bi/bi1)

plt.ylim(0.8, 1.1)
plt.axhline(1)

In [ ]:
dV = SNgenerator.cosmology.comoving_volume(a[1:]) - SNgenerator.cosmology.comoving_volume(a[:-1])
plt.plot((a[:-1] + a[1:]) * 0.5,
         bi/dV * np.sum(dV))
plt.plot((a[:-1] + a[1:]) * 0.5, n_obj * 1.08  / (1 + (a[:-1] + a[1:]) * 0.5))

In [ ]:
dz = (a[:-1] + a[1:]) * 0.5


In [ ]:
1  / (1 + dz[0]) - 1  / (1 + dz[-1])

In [ ]:
cosmology

In [ ]:
np.sum(1 / (1 + (a[:-1] + a[1:]) * 0.5)) * (a[1] - a[0])

In [ ]:
np.sqrt(740) 

In [ ]:
740 * 0.05

In [ ]:
plt.scatter(*survey.fields._sub_fields_corners[61].T)

In [6]:
import shapely as shp

In [ ]:
p = shp.geometry.Polygon(survey.fields._sub_fields_corners[61])

In [ ]:
shp.affinity.rotate(p, 45, 'center')

In [18]:
@nb.njit()
def R_base(a, t, vec, to_field_frame=True):
    """Return the new carthesian coordinates after z axis and vec axis rotations.
    Parameters
    ----------
    a : float
        Rotation angle around z axis.
    t : Rotation angle around vec axis.
    vec : numpy.ndarray(float)
        Coordinates of the second rotation axis.
    Returns
    -------
    numpy.ndarray(float)
        Carthesian coordinates in the new basis.
    Notes
    -----
    Rotation matrix computed using sagemaths
    """
    R = np.zeros((3, 3))
    R[0, 0] = (np.cos(t) - 1) * np.cos(a) * np.sin(a)**2 - \
        ((np.cos(t) - 1) * np.sin(a)**2 - np.cos(t)) * np.cos(a)
    R[0, 1] = (np.cos(t) - 1) * np.cos(a)**2 * np.sin(a) + \
        ((np.cos(t) - 1) * np.sin(a)**2 - np.cos(t)) * np.sin(a)
    R[0, 2] = np.cos(a) * np.sin(t)
    R[1, 0] = (np.cos(t) - 1) * np.cos(a)**2 * np.sin(a) - \
        ((np.cos(t) - 1) * np.cos(a)**2 - np.cos(t)) * np.sin(a)
    R[1, 1] = -(np.cos(t) - 1) * np.cos(a) * np.sin(a)**2 - \
        ((np.cos(t) - 1) * np.cos(a)**2 - np.cos(t)) * np.cos(a)
    R[1, 2] = np.sin(a) * np.sin(t)
    R[2, 0] = -np.cos(a)**2 * np.sin(t) - np.sin(a)**2 * np.sin(t)
    R[2, 1] = 0
    R[2, 2] = np.cos(t)

    if to_field_frame:
        return R.T @ vec
    else:
        return R @ vec

In [5]:
import numba as nb


In [19]:
@nb.guvectorize(["void(float64[:, :], float64[:, :], float64[:,:])"],
              "(m, n),(m, n)->(m, n)", nopython=True)
def new_coord_on_fields(ra_dec, ra_dec_frame, new_radec):
    """Compute new coordinates of an object in a list of fields frames.
    Parameters
    ----------
    ra_frame : numpy.ndarray(float)
        Field Right Ascension.
    dec_frame : numpy.ndarray(float)
        Field Declinaison.
    vec : numpy.ndarray(float, size = 3)
        The carthesian coordinates of the object.
    Returns
    -------
    numpy.ndarray(float, size = (2, ?))
    The new coordinates of the obect in each field frame.
    """
   
    for i in range(len(ra_dec_frame[0])):
        vec = np.array([np.cos(ra_dec[0][i]) * np.cos(ra_dec[1][i]),
                        np.sin(ra_dec[0][i]) * np.cos(ra_dec[1][i]),
                        np.sin(ra_dec[1][i])])
        x, y, z = R_base(ra_dec_frame[0][i], -ra_dec_frame[1][i], vec, to_field_frame=False)
        new_radec[0][i] = np.arctan2(y, x)
        if  new_radec[0][i] <0: new_radec[0][i] +=  2 * np.pi
        new_radec[1][i] = np.arcsin(z)


In [13]:
%%time
edges = np.array([np.ones(1000) * 2 * np.pi, np.linspace(-np.pi/2, np.pi/2, 1000)]).T
limit = shp.geometry.LineString(edges)
def comput_polygon(corner):
    polygon = shp.geometry.Polygon(corner)
    if polygon.intersects(limit):
        unioned = polygon.boundary.union(limit)
        polygon = [p for p in shp.ops.polygonize(unioned)
                   if p.representative_point().within(polygon)]
        x0, y0 = polygon[0].boundary.xy
        x1, y1 = polygon[1].boundary.xy
        if x1 > x0: 
            x1 = np.array(x1) - 2 * np.pi
            polygon[1] = shp.geometry.Polygon(np.array([x1, y1]).T)
        else:
            x0 = np.array(x0) - 2 * np.pi
            polygon[0] = shp.geometry.Polygon(np.array([x0, y0]).T)
        polygon =  shp.geometry.MultiPolygon(polygon)
    return polygon

CPU times: user 1.43 ms, sys: 925 µs, total: 2.36 ms
Wall time: 1.82 ms


In [ ]:
poly = shp.geometry.Polygon(vertices)

# If poly intersect edges cut it into 2 polygons
if poly.intersects(limit):
    unioned = poly.boundary.union(limit)
    poly = [p for p in shp.ops.polygonize(unioned)
            if p.representative_point().within(poly)]
    x, y = poly[0].boundary.xy
    x = np.array(x) - 2 * np.pi
    poly[0] = shp.geometry.Polygon(np.array([x, y]).T)    

In [ ]:
Jb = 1024

for i in range(4):
    plt.scatter(corner[i][0][Jb] - np.pi, corner[i][1][Jb], label=f'{i}', s=100) 
plt.legend()

In [ ]:
for i in range(4):
    c = f'C{i}'
    for j in range(64):
        ra = corner[i][0][Jb + j]
        dec = corner[i][1][Jb + j]
        if ra > np.pi: ra -= 2 * np.pi
        plt.scatter(ra, dec, label=f'{i}', s=4)
        
plt.scatter(testtable.fieldRA.values[Jb], testtable.fieldDec.values[Jb], marker='*')
plt.axvline(0, ls='--', c='k')
#plt.axvline(5.8, ls='--', c='k')
#plt.xlim(5.7, 6.5)

In [20]:
%%time
testtable = survey.obs_table.iloc[:1_000_000]

subfc = np.stack(testtable.rcid.map(survey.fields._sub_fields_corners).values)

corner = {}
for i in range(4):
    corner[i] = new_coord_on_fields(subfc[:, i].T, [testtable.fieldRA.values, testtable.fieldDec.values])


CPU times: user 3.86 s, sys: 213 ms, total: 4.07 s
Wall time: 4.07 s


In [21]:
%%time

sign = (corner[3][0] - testtable.fieldRA.values) * (corner[0][0] - testtable.fieldRA.values) < 0
comp = corner[0][0] < corner[3][0]

corner[1][0][corner[1][0] < corner[0][0]] += 2 * np.pi
corner[2][0][corner[2][0] < corner[3][0]] += 2 * np.pi


corner[0][0][sign & comp] += 2 * np.pi
corner[1][0][sign & comp] += 2 * np.pi

corner[2][0][sign & ~comp] += 2 * np.pi
corner[3][0][sign & ~comp] += 2 * np.pi

CPU times: user 18.2 ms, sys: 97 µs, total: 18.3 ms
Wall time: 16.6 ms


In [22]:
%%time
GeoS = gpd.GeoDataFrame(data=testtable, 
                        geometry=[comput_polygon([[corner[i][0][j], 
                                                   corner[i][1][j]] for i in range(4)]) 
                        for j in range(len(testtable))])

CPU times: user 25.9 s, sys: 1.37 s, total: 27.3 s
Wall time: 29.4 s


In [23]:
n=10
SNRA = np.random.uniform(0, 2 * np.pi, size=n)
SNDEC = np.random.uniform(- np.pi/2, np.pi/2, size=n)



In [24]:
geoarray = gpd.points_from_xy(*[SNRA, SNDEC])
geopoints = gpd.GeoDataFrame(geometry=geoarray)

In [28]:
%%time
TABLE = geopoints.sjoin(GeoS, how="inner", predicate="intersects",)

CPU times: user 607 ms, sys: 82.2 ms, total: 689 ms
Wall time: 1.26 s


In [30]:
TABLE

,geometry,index_right,expMJD,filter,fieldID,fieldRA,fieldDec,maglimcat,zp,gain,rcid,sig_zp,sig_psf
4,POINT (3.36459 0.97947),336918,58297.257812,ztfr,790,3.340386,0.959058,20.508080,26.181080,6.2,37,0.01,0.0
4,POINT (3.36459 0.97947),557509,58303.203125,ztfg,790,3.340386,0.959058,21.731619,26.127619,6.2,37,0.01,0.0
4,POINT (3.36459 0.97947),671502,58306.222656,ztfg,790,3.340386,0.959058,21.294973,25.944973,6.2,37,0.01,0.0
4,POINT (3.36459 0.97947),522806,58302.187500,ztfi,790,3.340386,0.959058,21.077644,25.632645,6.2,37,0.01,0.0
4,POINT (3.36459 0.97947),82016,58290.187500,ztfi,790,3.340386,0.959058,20.538584,25.615583,6.2,37,0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,POINT (5.28508 1.03251),220818,58293.429688,ztfg,828,5.235988,1.084722,21.374254,26.292255,6.2,7,0.01,0.0
9,POINT (5.28508 1.03251),496219,58301.289062,ztfg,828,5.235988,1.084722,20.736866,26.245865,6.2,7,0.01,0.0
9,POINT (5.28508 1.03251),950590,58317.417969,ztfr,828,5.235988,1.084722,20.764162,25.774160,6.2,7,0.01,0.0
9,POINT (5.28508 1.03251),346290,58297.328125,ztfr,828,5.235988,1.084722,20.388311,26.192310,6.2,7,0.01,0.0


In [ ]:
for i in range(4):
    c = f'C{i}'
    for j in range(64):
        ra = corner[i][0][Jb + j]
        dec = corner[i][1][Jb + j]
        plt.scatter(ra, dec, label=f'{i}', s=4)
        
plt.scatter(testtable.fieldRA.values[Jb], testtable.fieldDec.values[Jb], marker='*')
plt.axvline(2 * np.pi, ls='--', c='k')
#plt.axvline(5.8, ls='--', c='k')
#plt.xlim(5.7, 6.5)
plt.xlim(0, 1)


In [ ]:
Jb = 33404

for i in range(4):
    c = f'C{i}'
    for j in range(64):
        ra = corner[i][0][Jb + j]
        dec = corner[i][1][Jb + j]
        #if ra > np.pi: ra -= 2 * np.pi
        plt.scatter(ra, dec, label=f'{i}', s=4)
        
plt.scatter(testtable.fieldRA.values[Jb], testtable.fieldDec.values[Jb], marker='*')
plt.axvline(2 * np.pi, ls='--', c='k')
#plt.axvline(5.8, ls='--', c='k')
plt.xlim(5.7, 6.5)



In [ ]:
Polygon[29]

In [ ]:
x0, y0 = Polygon[30].geoms[0].boundary.xy
x1, y1 = Polygon[30].geoms[1].boundary.xy
plt.fill(x0, y0)
plt.fill(x1, y1)
plt.xlim(5.7, 2*np.pi)

In [ ]:
RAF = [survey.fields._dic[k]['ra'] for k in survey.fields._dic]
DECF = [survey.fields._dic[k]['dec'] for k in survey.fields._dic]

In [ ]:
def compute_sfield_corner(ra_dec_corner, RAF, DECF):
    ra_dec_fcorner = np.zeros((len(ra_dec_corner[0]), 2, len(RAF)))
    
    for i in range(len(ra_dec_corner[0])):
        rac, decc = new_coord_on_fields([ra_dec_corner[0][i],
                                         ra_dec_corner[1][i]],
                                        [RAF, DECF])
        ra_dec_fcorner[i][0][:] = rac
        ra_dec_fcorner[i][1][:] = decc
    return ra_dec_fcorner

In [ ]:
ra_dec_corner = survey.obs_table

In [ ]:
res = compute_sfield_corner(survey.fields._sub_fields_corners[61].T, RAF, DECF)

In [ ]:
shp.ops.unary_union(P)

In [ ]:
import geopandas as gpd

In [ ]:
idx = []
idx_tmp = np.random.choice(host.table.index, size=10)
samples = np.array([host.table.loc[idx_tmp]['ra'], host.table.loc[idx_tmp]['dec']]).T
multipoint = shp.geometry.MultiPoint(samples)
sbool = gpd.GeoSeries(multipoint).explode(index_parts=False)


In [ ]:
samples

In [ ]:
shp.geometry.Polygon(

In [ ]:
idx.extend(idx_tmp[sbool.within(host._footprint)])
n_to_sim = n - len(idx)

In [ ]:
plt.subplot(projection='mollweide')
plt.scatter(res[0][0][:], res[0][1][:],s=1)
plt.scatter(res[1][0][:], res[1][1][:],s=1)
plt.scatter(res[2][0][:], res[1][1][:],s=1)
plt.scatter(res[3][0][:], res[1][1][:],s=1)


In [ ]:
new_coord_on_fields

In [ ]:
ra_dec_frame[0][1]